In [ ]:
import pandas as pd
import numpy as np
from data_import import load_data
from model_dfs import prepare_nig_inputs
#from nig_em_paper import

In [2]:
# Load Accenture dataset
ret_daily, bs = load_data(
    xlsx_path= None,
    verbose=True
)

print(ret_daily.head())
print("-"*40)
print(bs.head())
print("-"*40)

# Load ECB 1Y risk-free yield data

# If first time, call API to get data, otherwise:
df_rf= pd.read_csv("ecb_riskfree_1y_daily.csv", parse_dates=["date"])
print(df_rf.head())

      country_iso          isin       date                       company  \
41651         DEU  DE0005190003 2010-01-05  BAYERISCHE MOTOREN WERKE AKT   
41652         DEU  DE0005190003 2010-01-06  BAYERISCHE MOTOREN WERKE AKT   
41653         DEU  DE0005190003 2010-01-07  BAYERISCHE MOTOREN WERKE AKT   
41654         DEU  DE0005190003 2010-01-08  BAYERISCHE MOTOREN WERKE AKT   
41655         DEU  DE0005190003 2010-01-11  BAYERISCHE MOTOREN WERKE AKT   

        gvkey   shares_out   close  mcap_reported  shares_out_filled  \
41651  100022  601995196.0  32.310   1.945046e+10        601995196.0   
41652  100022  601995196.0  32.810   1.975146e+10        601995196.0   
41653  100022  601995196.0  33.100   1.992604e+10        601995196.0   
41654  100022  601995196.0  32.655   1.965815e+10        601995196.0   
41655  100022  601995196.0  32.170   1.936619e+10        601995196.0   

               mcap  bad_day  logret_close  logret_mcap  
41651  1.945046e+10    False      0.008080     0.008

In [ ]:
# Build NIG inputs and fill missing liabilities
df_nig_panel, nig_em_data = prepare_nig_inputs(ret_daily, bs, df_rf)
print(df_nig_panel.head())


     gvkey       date             E          isin  \
0   100022 2010-01-05  1.945046e+10  DE0005190003   
1   100080 2010-01-05  4.578810e+10  DE000BAY0017   
2   100312 2010-01-05  1.765719e+09  DE0007030009   
3   100581 2010-01-05  4.701386e+10  FR0000120321   
4   100957 2010-01-05  3.539016e+10  ES0144580Y14   
5   101202 2010-01-05  2.188290e+10  FR0000120073   
6   101204 2010-01-05  7.372136e+10  FR0000120578   
7   101248 2010-01-05  8.674785e+09  FR0000121667   
8   101305 2010-01-05  6.061525e+09  FR0000073272   
9   101336 2010-01-05  2.149174e+10  FR0000121972   
10  101361 2010-01-05  4.566206e+09  NL0000395903   
11  102296 2010-01-05  2.096499e+10  FR0000125486   
12  103487 2010-01-05  4.002267e+10  DE0007164600   
13  132740 2010-01-05  4.605613e+09  DE0006231004   
14   14140 2010-01-05  9.784074e+10  ES0113900J37   
15   14447 2010-01-05  3.839475e+10  FR0000121014   
16   15181 2010-01-05  4.973802e+10  ES0113211835   
17   15532 2010-01-05  6.765080e+10  FR0000131

In [4]:
em_init = em_init_nig_params(
    equity=E,
    liabilities_L=L,
    rf=r,
    start_params=start_params,
    trading_days=250,
    pd_horizon_years=1.0,
    min_iter=3,
    max_iter=10,
    tol=1e-3,
)
params0 = em_init.params  # dict with alpha, beta1, delta, beta0
theta_series = em_init.theta_series  # risk-neutral tilts
asset_path = em_init.asset_path


In [5]:
# Check output sanity
print("L/E quantiles:",
      float((L/E).min()),
      np.quantile(L/E, [0.1, 0.5, 0.9]),
      float((L/E).max()))


E = np.asarray(E, float)
r = np.asarray(r, float)
A = np.asarray(em_init.asset_path, float)

print("converged:", em_init.converged, "n_iter:", em_init.n_iter, "diff_last:", em_init.diff_last)
print("params:", em_init.params)

# Inversion fallback check (critical)
fallback = np.isclose(A, E + L, rtol=1e-8, atol=1e-10)
print("fallback rate A≈E+L:", float(fallback.mean()))

# Structural sanity
print("share(A < E):", float(np.mean(A < E)))
print("min(A/E):", float(np.min(A / E)))

# Vol sanity (equity typically more volatile than assets in structural models)
rE = np.diff(np.log(E))
rA = np.diff(np.log(A))
print("std(rE):", float(np.std(rE)), "std(rA):", float(np.std(rA)))


alpha = em_init.params["alpha"]
beta1 = em_init.params["beta1"]
theta = np.asarray(em_init.theta_series, float)

beta_minus = beta1 + theta
beta_plus  = beta1 + theta + 1.0

print("max|beta_minus|:", float(np.max(np.abs(beta_minus))), "alpha:", float(alpha))
print("max|beta_plus| :", float(np.max(np.abs(beta_plus ))), "alpha:", float(alpha))
print("pricing feasible all days?:",
      (np.max(np.abs(beta_minus)) < alpha) and (np.max(np.abs(beta_plus)) < alpha))


from scipy.stats import norminvgauss, norm
import numpy as np

p = em_init.params
alpha, beta, delta, mu = p["alpha"], p["beta1"], p["delta"], p["beta0"]
h = 1/250

rA = np.diff(np.log(em_init.asset_path))

ll_nig = np.sum(norminvgauss.logpdf(rA, alpha, beta, loc=mu*h, scale=delta*h))
ll_norm = np.sum(norm.logpdf(rA, loc=np.mean(rA), scale=np.std(rA)))

print("loglik NIG:", float(ll_nig))
print("loglik Normal:", float(ll_norm))


L/E quantiles: 0.1557077024216854 [0.16561023 0.21426602 0.24053481] 0.2615566918475622
converged: True n_iter: 4 diff_last: [2.09950459e-05 1.98392548e-05 3.47998011e-05 3.53858054e-06]
params: {'alpha': 0.6395072585666938, 'beta1': -0.044855228893943964, 'delta': 2.856675170693696, 'beta0': 0.33215798623342496}
fallback rate A≈E+L: 0.0
share(A < E): 0.0
min(A/E): 1.1826527291827127
std(rE): 0.01637388064619858 std(rA): 0.014622298447226305
max|beta_minus|: 0.6263181473157983 alpha: 0.6395072585666938
max|beta_plus| : 0.3751646011007852 alpha: 0.6395072585666938
pricing feasible all days?: True
loglik NIG: 1468.2319445420496
loglik Normal: 1414.3596220050435


##### 1. Units are coherent
L/E median ≈ 0.214 (quantiles ~0.16–0.26). That’s economically plausible for “liabilities vs market cap”. That confirms the scaling fix worked and equity/liabilities are now in the same units, which is required by the structural identity (equity as a call on assets with strike L).

##### 2) The asset inversion is working 
fallback rate A≈E+L: 0.0 is the big one. It is correctly solving C_NIG(A)=E each day.

##### 3) Structural/economic sanity checks pass
share(A<E)=0 and min(A/E)=1.18 → equity is always below assets (as it should be in a limited-liability structural model).
std(rA)=0.0146 < std(rE)=0.0164 → assets are less volatile than equity (equity is the levered/option-like claim). This is what you expect in Merton-style structural calibration.

##### 4) Pricing feasibility is satisfied
verified:
max|beta_minus| < alpha and max|beta_plus| < alpha → the shifted NIG parameters used in call pricing are valid for SciPy’s norminvgauss CDF, so the pricing equation is well-defined across the whole window.

##### 5) The “non-normality improvement” visible
loglik NIG = 1468.2 vs loglik Normal = 1414.4 on inferred asset returns → NIG fits materially better than Gaussian, which is aligned with the point of extending the Merton model beyond normality.